In [3]:
import requests
import numpy as np
import plots as pl
import pandas as pd
import matplotlib.pyplot as plt 
import aplpy 
import os
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
from IPython.core.display import HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import glob
from astropy.coordinates import ICRS, Galactic, FK4, FK5
from astropy import units as u
from astropy.coordinates import SkyCoord

/Users/134393/anaconda3/lib/python3.7/site-packages/mpl_toolkits/axes_grid/__init__.py:12: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid module was deprecated in Matplotlib 2.1 and will be removed two minor releases later. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist, which provide the same functionality instead.
  obj_type='module')


In [2]:
def get_image(source_id, big=None):
    image_all = filtered_raw_data[filtered_raw_data.source_id == source_id].sort_values(by='jd').image
    image_list = list(image_all)
    for im in range(len(image_list)):
        plt.figure(figsize=[10,12])
        image = image_list[im]
        s_stats = stats[stats.source_id == source_id]
        grid = True
        if big:
           size=(1200, 1200)
           im_rad = 8.0
        else: 
           size=(1200, 1200)
           im_rad = 1.0
        dpi = 40
        North = False # False - faster with image warping. True - slower but with less image warping. 
        figure = aplpy.FITSFigure('/Volumes/BEL27A-BKP01/MWATS/'+image, dimensions=[0,1], north=False, slices=[0,0], auto_refresh=False, dpi = dpi, figsize = (size[0] / float(dpi), size[1] / float(dpi)))
        ra = s_stats.ra.values[0]
        dec = s_stats.dec.values[0]
        circle_size = 1.0 # 1 degree circle
        figure.recenter(ra, dec, radius=im_rad, width=2.0, height=2.0)
        #figure.recenter(ra, dec, circle_size) 
        if grid:
            try:
                figure.add_grid()
            except Exception as e:
                logging.warn("APLpy error adding grid to image: {0}".format(e))
        # Adjust the axis labels, otherwise they overlap
        # If the image is less than two arc minutes across, show seconds on the tick labels
    
        image_radius = circle_size
        if image_radius < 1.0 / 60.0 * 2:
                figure.tick_labels.set_xformat("hh:mm:ss")
                figure.tick_labels.set_yformat("dd:mm:ss")
        else:
                figure.tick_labels.set_xformat("hh:mm")
                figure.tick_labels.set_yformat("dd:mm")

        figure.tick_labels.set_xformat("dd:mm")
        figure.tick_labels.set_yformat("dd:mm")
        figure.show_grayscale(pmin=5.0, pmax=98, invert=False)
        #figure.add_colorbar()
        # Show locations of the neighbours
        figure.show_rectangles(ra, dec, 0.1, 0.1, edgecolor='k', linewidth=2.0)
        n = neighbours[neighbours.source_id == source_id]
        for num in range(5):
            if num == 0:
               neigh = n.n_id1.values[0]
               nra  =  n.n_id1_ra.values[0]
               ndec =  n.n_id1_dec.values[0]
               colour = 'm'
            if num == 1:
               neigh = n.n_id2.values[0]
               nra  =  n.n_id2_ra.values[0]
               ndec =  n.n_id2_dec.values[0]
               colour = 'y'
            if num == 2:
               neigh = n.n_id3.values[0]
               nra  =  n.n_id3_ra.values[0]
               ndec =  n.n_id3_dec.values[0]
               colour = 'c'
            if num == 3:
               neigh = n.n_id4.values[0]
               nra  =  n.n_id4_ra.values[0]
               ndec =  n.n_id4_dec.values[0]
               colour = 'b'
            if num == 4:
               neigh = n.n_id5.values[0]
               nra  =  n.n_id5_ra.values[0]
               ndec =  n.n_id5_dec.values[0]
               colour = 'r'
            figure.show_ellipses(nra, ndec, 0.1, 0.1, edgecolor=colour, linewidth=2.0)
        figure.axis_labels.set_font(size = 30.0)
        figure.tick_labels.set_font(size = 30.0)
        figure.add_colorbar()
        figure.colorbar.set_font(size = 30.0)
        figure.colorbar.set_axis_label_font(size = 30.0)
        figure.colorbar.set_axis_label_text('Jy / beam')
        if im <= 9:
           path = 'movies/00'+str(im)
        elif im > 99:
           path = 'movies/'+str(im)
        else:
           path = 'movies/0'+str(im)
        figure.save(path+'_a.png')
        figure.close()
        plt.close()
        

In [3]:
def plot_n_lc(source):
  main = filtered_raw_data[filtered_raw_data.source_id == source]
  main = main.set_index('time')
  main = main.sort_values(by='jd')
  for frame in range(len(main.jd)):
    plt.figure(figsize=[8,6])
    gs = gridspec.GridSpec(3, 2)
    n = neighbours[neighbours.source_id == source]
    for num in range(5):
        if num == 0:
               neigh = n.n_id1.values[0]
               colour = 'm'
        if num == 1:
               neigh = n.n_id2.values[0]
               colour = 'y'
        if num == 2:
               neigh = n.n_id3.values[0]
               colour = 'c'
        if num == 3:
               neigh = n.n_id4.values[0]
               colour = 'b'
        if num == 4:
               neigh = n.n_id5.values[0]
               colour = 'r'
        plt.subplot(gs[num])    
        s_stats = stats[stats.source_id == source]
        n = neighbours[neighbours.source_id == source]
        n_data = filtered_raw_data[filtered_raw_data.source_id == neigh]
        n_data = n_data.sort_values(by='jd')
        n_data = n_data.set_index('time')
        lc1_lc2 = pd.merge(main, n_data, on='time', how='left')
 
        plt.plot(range(frame), lc1_lc2.raw_peak_flux_y[0:frame], color=colour, markersize=5, marker='.', label='Neighbour', alpha=0.5)
        plt.plot(range(frame), main.raw_peak_flux[0:frame], 'ko-.', label='Main', alpha=0.5)
        plt.xlim(0,len(main.jd))
        plt.ylabel('Flux (Jy)')
        plt.xlabel('Image sequence (N)')
    
    # Plot the gain
    plt.subplot(gs[5])
    Gs = list(main.Gain)
    Gs = Gs[0:frame]
    plt.plot(Gs, 'ko', alpha=0.5)
    plt.xlim(0,len(main.Gain))
    plt.ylabel('Gain')
    plt.xlabel('Image sequence (N)')
    plt.tight_layout()
    
    ######## SAVE #######
    if frame <= 9:
     path = 'movies/00'+str(frame)
    elif frame > 99:
     path = 'movies/'+str(frame)
    else:
     path = 'movies/0'+str(frame)
    plt.savefig(path+'_b.png')
    plt.close()

In [1]:
def combine():
  for image in glob.glob('/Users/134393/Work/MWATS_Full/mwats/SQL/movies/*_a.png'):
    im2 = image.replace('_a.png','_b.png')
    img=mpimg.imread(image)
    img2=mpimg.imread(im2)
    plt.figure(figsize=[20,10])
    plt.subplot(1,2,1)
    fig1 = plt.imshow(img)
    plt.subplot(1,2,2)
    fig2 = plt.imshow(img2)
    fig1.axes.get_xaxis().set_visible(False)
    fig1.axes.get_yaxis().set_visible(False)
    fig2.axes.get_xaxis().set_visible(False)
    fig2.axes.get_yaxis().set_visible(False)
    plt.tight_layout()
    plt.savefig(image.replace('_a.png','.png'), dpi=400)

In [5]:
# Load the data
stats = pd.read_feather('stats_table.fth')
filtered_raw_data = pd.read_feather('data.fth')
neighbours = pd.read_feather('neighbours.fth')

In [ ]:
os.chdir('/Users/134393/Work/MWATS_Full/mwats/SQL/')
source_id = 42516
bigit=False
os.system('rm /Users/134393/Work/MWATS_Full/mwats/SQL/movies/*.png')
plot_n_lc(source_id)
get_image(source_id, big=bigit)
combine()

INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -3.631e-01 [aplpy.core]
INFO: Auto-setting vmax to  7.072e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -3.698e-01 [aplpy.core]
INFO: Auto-setting vmax to  7.153e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -3.812e-01 [aplpy.core]
INFO: Auto-setting vmax to  7.351e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -3.747e-01 [aplpy.core]
INFO: Auto-setting vmax to  7.217e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -3.927e-01 [aplpy.core]
INFO: Auto-setting vmax to  7.533e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -3.819e-01 [aplpy.core]
INFO: Auto-setting vmax to  7.322e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -4.225e-01 [aplpy.core]
INFO: Auto-setting vmax to  8.079e-01 [aplpy.core]

INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -5.173e-01 [aplpy.core]
INFO: Auto-setting vmax to  9.756e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -5.236e-01 [aplpy.core]
INFO: Auto-setting vmax to  9.955e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -5.123e-01 [aplpy.core]
INFO: Auto-setting vmax to  9.678e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -4.181e-01 [aplpy.core]
INFO: Auto-setting vmax to  8.046e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -4.437e-01 [aplpy.core]
INFO: Auto-setting vmax to  8.458e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -4.344e-01 [aplpy.core]
INFO: Auto-setting vmax to  8.279e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -4.423e-01 [aplpy.core]
INFO: Auto-setting vmax to  8.405e-01 [aplpy.core]

INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -3.997e-01 [aplpy.core]
INFO: Auto-setting vmax to  7.600e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -3.894e-01 [aplpy.core]
INFO: Auto-setting vmax to  7.426e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -3.984e-01 [aplpy.core]
INFO: Auto-setting vmax to  7.628e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -3.752e-01 [aplpy.core]
INFO: Auto-setting vmax to  7.126e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -4.111e-01 [aplpy.core]
INFO: Auto-setting vmax to  7.796e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -3.057e-01 [aplpy.core]
INFO: Auto-setting vmax to  5.915e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -3.179e-01 [aplpy.core]
INFO: Auto-setting vmax to  6.070e-01 [aplpy.core]

INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -5.089e-01 [aplpy.core]
INFO: Auto-setting vmax to  9.626e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -5.835e-01 [aplpy.core]
INFO: Auto-setting vmax to  1.086e+00 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -4.398e-01 [aplpy.core]
INFO: Auto-setting vmax to  8.317e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -4.526e-01 [aplpy.core]
INFO: Auto-setting vmax to  8.644e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -4.465e-01 [aplpy.core]
INFO: Auto-setting vmax to  8.518e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -4.388e-01 [aplpy.core]
INFO: Auto-setting vmax to  8.359e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -4.549e-01 [aplpy.core]
INFO: Auto-setting vmax to  8.648e-01 [aplpy.core]

/Users/134393/anaconda3/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [ ]:
os.chdir('/Users/134393/Work/MWATS_Full/mwats/SQL/movies/')
os.system('ffmpeg -framerate 10 -i %03d.png -s:v 1280x720 -c:v libx264 -profile:v high -crf 1 -pix_fmt yuv420p /Users/134393/Work/MWATS_Full/mwats/SQL/movies/'+str(source_id)+'.mp4')

In [ ]:
combine()

/Users/134393/anaconda3/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
